In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-cse-uoi-2025/data_new/abstracts.txt
/kaggle/input/nlp-cse-uoi-2025/data_new/edgelist.txt
/kaggle/input/nlp-cse-uoi-2025/data_new/test.txt
/kaggle/input/nlp-cse-uoi-2025/data_new/submission_random.csv
/kaggle/input/nlp-cse-uoi-2025/data_new/authors.txt


In [2]:
import nltk

# Download necessary NLTK resources 
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("averaged_perceptron_tagger")
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

5 cross validation add

In [3]:
import networkx as nx

since we got seperations in txt we are gonna investigate the contents of each .txt . seperations did not work so we split with regular expression r....| -- | name it ID int column and abstract then create a pd

In [4]:
#other imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Imports and Configuration
This cell imports all necessary Python libraries for data manipulation, machine learning, and utility functions.
It also defines file paths for datasets from /kaggle

In [5]:
import pandas as pd
import numpy as np
import io
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import random
import time 

PATH_EDGELIST = '/kaggle/input/nlp-cse-uoi-2025/data_new/edgelist.txt' 
PATH_AUTHORS = '/kaggle/input/nlp-cse-uoi-2025/data_new/authors.txt'   
PATH_ABSTRACTS = '/kaggle/input/nlp-cse-uoi-2025/data_new/abstracts.txt' 
PATH_TEST_SET = '/kaggle/input/nlp-cse-uoi-2025/data_new/test.txt'      
PATH_SAMPLE_SUBMISSION = '/kaggle/input/nlp-cse-uoi-2025/data_new/submission_random.csv'

# Data Loading Functions
This cell defines several simple def functions to load and parse your raw text data files: edgelist.txt (citation links), authors.txt (author information), abstracts.txt (paper abstracts), and test.txt (pairs for prediction). Each function includes basic error handling and prints the time taken for loading.

In [6]:
def load_edgelist(file_path):
    start_time = time.time()
    try:
        df = pd.read_csv(file_path, header=None, names=['citing_paper_id', 'cited_paper_id'])
        print(f"Successfully loaded edgelist: {len(df)} edges from {file_path} (took {time.time() - start_time:.2f}s)")
        return df
    except FileNotFoundError:
        print(f"Error: Edgelist file not found at {file_path}. Please check the path.")
        return pd.DataFrame(columns=['citing_paper_id', 'cited_paper_id'])



In [7]:
def load_authors_data(file_path):
    start_time = time.time()
    paper_ids = []
    authors_lists = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                parts = line.strip().split('|--|')
                if len(parts) == 2:
                    paper_ids.append(parts[0])
                    authors = [author.strip() for author in parts[1].split(',')]
                    authors_lists.append(authors)
        df_authors_loaded = pd.DataFrame({'paper_id': paper_ids, 'authors': authors_lists})
        df_authors_loaded['paper_id'] = pd.to_numeric(df_authors_loaded['paper_id'], errors='coerce')
        df_authors_loaded.dropna(subset=['paper_id'], inplace=True)
        df_authors_loaded['paper_id'] = df_authors_loaded['paper_id'].astype(int)
        print(f"Succes loading and parsed authors data: {len(df_authors_loaded)} entries from {file_path} (took {time.time() - start_time:.2f}s)")
        return df_authors_loaded
    except FileNotFoundError:
        print(f"Error1: Authors file not found at {file_path}. Please check the path.")
        return pd.DataFrame(columns=['paper_id', 'authors'])
    except Exception as e:
        print(f"Error2 processing authors file {file_path}: {e}")
        return pd.DataFrame(columns=['paper_id', 'authors'])

In [8]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    # replace what  not a letter or number with a space
    text = re.sub(r'\W+', ' ', text)
    words = text.split()
    # creates a new list of words -> if they are not a stopword, and after lemmatizing them
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # joins the list of words to a single string with spaces  ' '
    return ' '.join(words)

def load_abstracts_data(file_path):
    start_time = time.time()
    paper_ids = []
    abstract_texts = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                parts = line.strip().split('|--|', 1)
                if len(parts) == 2:
                    paper_ids.append(parts[0])
                    abstract_texts.append(parts[1])
        df_abstracts_loaded = pd.DataFrame({'paper_id': paper_ids, 'abstract': abstract_texts})
        
        df_abstracts_loaded['paper_id'] = pd.to_numeric(df_abstracts_loaded['paper_id'], errors='coerce')
        df_abstracts_loaded.dropna(subset=['paper_id'], inplace=True)
        df_abstracts_loaded['paper_id'] = df_abstracts_loaded['paper_id'].astype(int)
        
        # cc the abstracts
        df_abstracts_loaded['clean_abstract'] = df_abstracts_loaded['abstract'].apply(clean_text)

        print(f"Successfull load and parse of abstracts data: {len(df_abstracts_loaded)} entries from {file_path} (took {time.time() - start_time:.2f}s)")
        return df_abstracts_loaded
    except FileNotFoundError:
        print(f"Error1: Abstracts file not found at {file_path}. Please check the path again.")
        return pd.DataFrame(columns=['paper_id', 'abstract'])
    except Exception as e:
        print(f"Error2 processing abstracts file {file_path}: {e}")
        return pd.DataFrame(columns=['paper_id', 'abstract'])


In [9]:
def load_test_pairs(file_path):
    start_time = time.time()
    try:
        df= pd.read_csv(file_path, header=None, names=['paper_id_1', 'paper_id_2'])
        print(f"Successfully loaded the needed test pairs: {len(df)} pairs from {file_path} (took {time.time() - start_time:.2f}s)")
        return df
    except FileNotFoundError:
        print(f"Error1: Test pairs file not found at {file_path}. Please check the path.")
        return pd.DataFrame(columns=['paper_id_1', 'paper_id_2'])


loading done proceding to see df

# Load Main Datasets
This cell executes the loading functions defined previously to load all datasets into pandas DataFrames. It then prints the shapes of these DataFrames to verify they've been loaded correctly.

In [10]:
df_edgelist = load_edgelist(PATH_EDGELIST)
df_authors = load_authors_data(PATH_AUTHORS)
df_abstracts = load_abstracts_data(PATH_ABSTRACTS)
df_test_kaggle = load_test_pairs(PATH_TEST_SET)



Successfully loaded edgelist: 1091955 edges from /kaggle/input/nlp-cse-uoi-2025/data_new/edgelist.txt (took 0.26s)
Succes loading and parsed authors data: 138499 entries from /kaggle/input/nlp-cse-uoi-2025/data_new/authors.txt (took 0.56s)
Successfull load and parse of abstracts data: 138499 entries from /kaggle/input/nlp-cse-uoi-2025/data_new/abstracts.txt (took 54.01s)
Successfully loaded the needed test pairs: 106692 pairs from /kaggle/input/nlp-cse-uoi-2025/data_new/test.txt (took 0.05s)



**1 df_edgelist
2 df_authors
3 df_abstracts
4 df_test_kaggle**

**get information about each one of them and perform some more cleaning**

In [11]:
print(df_edgelist.head())
print("Shape")
print(df_edgelist.shape)
print("Info")
print(df_edgelist.info())

   citing_paper_id  cited_paper_id
0                0               1
1                0               2
2                1               3
3                1               5
4                1               6
Shape
(1091955, 2)
Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1091955 entries, 0 to 1091954
Data columns (total 2 columns):
 #   Column           Non-Null Count    Dtype
---  ------           --------------    -----
 0   citing_paper_id  1091955 non-null  int64
 1   cited_paper_id   1091955 non-null  int64
dtypes: int64(2)
memory usage: 16.7 MB
None


In [12]:
print(df_authors.head())
print("Shape")
print(df_authors.shape)
print("Info")
print(df_authors.info())

   paper_id                                            authors
0         0  [James H. Niblock, Jian-Xun Peng, Karen R. McM...
1         1          [Jian-Xun Peng, Kang Li, De-Shuang Huang]
2         2                                      [J. Heikkila]
3         3    [L. Teslic, B. Hartmann, O. Nelles, I. Skrjanc]
4         4  [Long Zhang, Kang Li, Er-Wei Bai, George W. Ir...
Shape
(138499, 2)
Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138499 entries, 0 to 138498
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   paper_id  138499 non-null  int64 
 1   authors   138499 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.1+ MB
None


In [13]:
print(df_abstracts.head())
print("Shape")
print(df_abstracts.shape)
print("Info")
print(df_abstracts.info())

   paper_id                                           abstract  \
0         0  The development of an automated system for the...   
1         1  This paper proposes a novel hybrid forward alg...   
2         2  Modern CCD cameras are usually capable of a sp...   
3         3  This paper deals with the problem of fuzzy non...   
4         4  A number of neural networks can be formulated ...   

                                      clean_abstract  
0  development automated system quality assessmen...  
1  paper proposes novel hybrid forward algorithm ...  
2  modern ccd camera usually capable spatial accu...  
3  paper deal problem fuzzy nonlinear model ident...  
4  number neural network formulated linear parame...  
Shape
(138499, 3)
Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138499 entries, 0 to 138498
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   paper_id        138499 non-null  int64 
 1   

In [14]:
print(df_test_kaggle.head())
print("Shape")
print(df_test_kaggle.shape)
print("Info")
print(df_test_kaggle.info())

   paper_id_1  paper_id_2
0       34977       59394
1       22518       46602
2       36762       22813
3       44960      110384
4       29015       26366
Shape
(106692, 2)
Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106692 entries, 0 to 106691
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   paper_id_1  106692 non-null  int64
 1   paper_id_2  106692 non-null  int64
dtypes: int64(2)
memory usage: 1.6 MB
None


# Negative Sampling Function
This cell defines the generate_negative_samples function. This function is crucial for creating a balanced training dataset by generating pairs of papers that do not have a citation link, complementing the positive examples (actual citations) from edgelist.txt.

In [15]:
def generate_negative_samples(edgelist_df, num_negative_factor=1):
    #generates negative samples
    start_time = time.time()
    if edgelist_df.empty:
        print("Edgelist is empty  [ ]. Cannot generate negative samples -/- .")
        return pd.DataFrame(columns=['paper_id_1', 'paper_id_2', 'target'])

    
    all_paper_ids_arr = pd.unique(edgelist_df[['citing_paper_id', 'cited_paper_id']].values.ravel('K'))
    
    if len(all_paper_ids_arr) == 0:
        print("No unique - paper IDs found in edgelist. Cannot generate negative samples -/- call.")
        return pd.DataFrame(columns=['paper_id_1', 'paper_id_2', 'target'])
    
    existing_edges= set(zip(edgelist_df['citing_paper_id'], edgelist_df['cited_paper_id']))

    
    num_positive_samples = len(edgelist_df)
    num_negative_samples_to_generate = int(num_positive_samples * num_negative_factor)
    
    negative_samples_list = []
    attempts= 0
    runs = 0
    attempts_limit= 0
    max_attempts = num_negative_samples_to_generate * 20 

    
    print(f"Attempting to generate {num_negative_samples_to_generate} negative samples..")
    while len(negative_samples_list) < num_negative_samples_to_generate and attempts < max_attempts:
        attempts += 1
        p1 = random.choice(all_paper_ids_arr)
        p2 = random.choice(all_paper_ids_arr)
        
        if p1 != p2 and (p1, p2) not in existing_edges:
            negative_samples_list.append({'paper_id_1': p1, 'paper_id_2': p2, 'target': 0})
            existing_edges.add((p1, p2)) 
        
        if attempts % (max_attempts // 100 + 1) == 0 : 
             print(f"  Negative sampling: {len(negative_samples_list)}/{num_negative_samples_to_generate} found after {attempts} attempts...", end='\r')

    print(f"\nGenerated {len(negative_samples_list)} negative samples from {attempts} attempts (took {time.time() - start_time:.2f}s).")
    if len(negative_samples_list) < num_negative_samples_to_generate:
        print(f"Warnng: run coulould only generate {len(negative_samples_list)} negative samples, requested {num_negative_samples_to_generate}.")
    return pd.DataFrame(negative_samples_list)


# Generate Negative Samples And Create Training Set
This cell calls the generate_negative_samples function above m using the loaded df_edgelist. It then combines these generated negative samples with the positive samples (actual citations given) to create the final df_train DataFrame, which is then stacked randomly.

In [16]:
df_negative_samples = generate_negative_samples(df_edgelist)

if not df_edgelist.empty:
    df_positive_samples = df_edgelist.copy()
    df_positive_samples.rename(columns={'citing_paper_id': 'paper_id_1', 'cited_paper_id': 'paper_id_2'}, inplace=True)
    
    df_positive_samples['target'] = 1


    
    df_train = pd.concat([df_positive_samples, df_negative_samples], ignore_index=True)
    df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print(f"Combined training data: {len(df_train)} samples.")
    print("Processed training data head below:\n", df_train.head())
else:
    df_train = pd.DataFrame()
    print("df_edgelist was empty, so df_train is empty. Continue non generation")


Attempting to generate 1091955 negative samples..
  Negative sampling: 1091862/1091955 found after 1091960 attempts...
Generated 1091955 negative samples from 1092053 attempts (took 2.71s).
Combined training data: 2183910 samples.
Processed training data head below:
    paper_id_1  paper_id_2  target
0       16646       73007       1
1       12798       44868       0
2        3212       17042       1
3      103632       71311       0
4       12509       71431       1


the result is acceptable as often the generated negative samples are roughly the same number of negatives

# Feature Engineering - Setup  and Helper Dictionaries
This cell begins the feature engineering process. It prepares several  dictionaries from the authors and abstracts data for efficient feature calculation. We used the tf - idf but we did not get acceptable results. We proceed with loading BERT SentenceTransformer model. The specific model is all-MiniLM-L6-v2.

s bert load

In [17]:
from sentence_transformers import SentenceTransformer 

# similarity using BERT embeddings
from sklearn.metrics.pairwise import cosine_similarity # 4.4 import cosine_similarity

print("Loading BERT SentenceTransformer model...") 
bert_model_load_start_time = time.time()
bert_model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"BERT SentenceTransformer model loaded (took {time.time() - bert_model_load_start_time:.2f}s)") # load time

# abstract similarity with bert 
def calculate_abstract_similarity_bert(row, embeddings_matrix, id_to_idx_map): # new similarity bert
    if embeddings_matrix is None: return 0.0
    idx1 = id_to_idx_map.get(row['paper_id_1'])
    
    idx2 = id_to_idx_map.get(row['paper_id_2'])
    if idx1 is not None and idx2 is not None and \
       idx1 < embeddings_matrix.shape[0] and idx2 < embeddings_matrix.shape[0]:
        # reshape(1, -1) for a 2d similarity array
        return cosine_similarity(embeddings_matrix[idx1].reshape(1, -1), embeddings_matrix[idx2].reshape(1, -1))[0, 0]
    return 0.0

2025-06-09 14:43:41.444581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749480221.622244      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749480221.678706      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading BERT SentenceTransformer model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

BERT SentenceTransformer model loaded (took 8.68s)


At that part of the code we were using TF-IDFand switched to BERT embeddings. We did a one time TF-IDF calculation and re-used it when needed. 

In [18]:
print("\nstarting feature enginering setup..")
start_fe_time = time.time()

authors_dict= df_authors.set_index('paper_id')['authors'].to_dict() if not df_authors.empty else {}
abstracts_map = df_abstracts.set_index('paper_id')['abstract'].to_dict() if not df_abstracts.empty else {}

abstract_embeddings = None #  BERT embeddings
pid_to_bert_idx = {} # kaps ver 4.2 map index BERT

if not df_abstracts.empty and abstracts_map: # available abstracts
    #  df_abstracts['clean_abstract'] 
    # available  clean_text() ?
    valid_abstract_paper_ids = [pid for pid in df_abstracts['paper_id'].unique() if pid in abstracts_map] # list confirmed IDs
    
    if valid_abstract_paper_ids: # valid- IDs
        corpus_for_embedding = [abstracts_map[pid] for pid in valid_abstract_paper_ids] # corpus prepa
        
        print("  Calculating BERT embeddings for abstracts ... (recommend gpu usage)") 
        bert_embed_start_time = time.time() 
        abstract_embeddings = bert_model.encode(corpus_for_embedding, show_progress_bar=True) # calculate BERT embeddings
        pid_to_bert_idx = {pid: i for i, pid in enumerate(valid_abstract_paper_ids)} # create map ID to index
        print(f"  BERT embeddings shape: {abstract_embeddings.shape} (took {time.time() - bert_embed_start_time:.2f}s)") #shape & time
    else:
        print("No valid abstracts to build BERT embeddings from (valid_abstract_paper_ids  is empty).") # not valid abstracts
else:
    print("Abstracts data has run out or abstracts_map is empty. Skipping BERT embeddings.") 

out_degrees = {}
in_degrees = {}
cited_by_map = defaultdict(set) 
out_degrees = {}
in_degrees = {}
cited_by_map = defaultdict(set) 
cites_map = defaultdict(set)
G_undirected = nx.Graph()

if not df_edgelist.empty:
    print("  Calculating graph-based features (degrees, maps, )...")
    graph_features_start_time = time.time()

    # calculate out-degrees (number of papers a paper cites)
    out_degrees = df_edgelist['citing_paper_id'].value_counts().to_dict()

    # calculate in-degrees (number of times a paper is cited)
    in_degrees = df_edgelist['cited_paper_id'].value_counts().to_dict()

    # create map of successors (papers cited by a given paper)
    cited_by_map_temp = df_edgelist.groupby('citing_paper_id')['cited_paper_id'].apply(set).to_dict()
    cited_by_map = defaultdict(set, cited_by_map_temp)

    # create map of predecessors (papers that cite a given paper)
    cites_map_temp = df_edgelist.groupby('cited_paper_id')['citing_paper_id'].apply(set).to_dict()
    cites_map = defaultdict(set, cites_map_temp)
    
    # Create the undirected graph for networkx functions
    G_undirected.add_edges_from(df_edgelist.values)

    print(f"  Done calculating graph features (took {time.time() - graph_features_start_time:.2f}s)")
else:
    print("Edgelist empty, so graph-based features will be 0 or based on empty maps.")

print(f"Total Feature Engineering Setup took {time.time() - start_fe_time:.2f}s")


starting feature enginering setup..
  Calculating BERT embeddings for abstracts ... (recommend gpu usage)


Batches:   0%|          | 0/4329 [00:00<?, ?it/s]

  BERT embeddings shape: (138499, 384) (took 182.96s)
  Calculating graph-based features (degrees, maps, )...
  Done calculating graph features (took 6.98s)
Total Feature Engineering Setup took 190.19s


# Feature Calculation Functions
This cell defines the individual functions used to calculate each feature: calculate_common_authors (number of shared authors)and calculate_jaccard_common_successors (Jaccard index of papers cited by both papers in a pair). The rest above in similiraty with BERT. We put them in the comment section and we run them with lambda in the engineer_features function below.

In [19]:
# def calculate_common_authors(row, authors_lookup):
#     p1_authors = set(authors_lookup.get(row['paper_id_1'], []))
#     p2_authors = set(authors_lookup.get(row['paper_id_2'], []))
#     return len(p1_authors.intersection(p2_authors))


# def calculate_jaccard_common_successors(row, cited_lookup_map):
#     p1_cites = cited_lookup_map.get(row['paper_id_1'], set()) 
#     p2_cites = cited_lookup_map.get(row['paper_id_2'], set()) 
    
#     intersection_size = len(p1_cites.intersection(p2_cites))
#     union_size = len(p1_cites.union(p2_cites))
    
#     return intersection_size / union_size if union_size > 0 else 0.0

# Main engineer_features Function
This cell defines the main engineer_features function. This function takes a DataFrame (either training or test data) and applies all the previously defined feature calculation functions and dictionary lookups to add new feature columns to the DataFrame.

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict 


def engineer_features(df, authors_lookup, abstract_embeddings_matrix, pid_to_embed_idx, out_deg_map, in_deg_map, cited_by_lookup_map, cites_lookup_map, nx_graph_undirected):

    #Applies all feature engineering step discussed 
    #including BERT similarity and advanced graph features.

    if df.empty:
        print("Input DataFrame is empty.")
        return df

    df_name = [name for name, var_val in globals().items() if var_val is df][0]
    print(f"\nEngineering features for DataFrame '{df_name}' with {len(df)} rows...")
    current_time = time.time()
    df_copy = df.copy()

    # 1. Common Authors
    print(" Calculating common authors...")
    df_copy['common_authors'] = df_copy.apply(lambda row: len(set(authors_lookup.get(row['paper_id_1'], [])).intersection(set(authors_lookup.get(row['paper_id_2'], [])))), axis=1)

    
    # 2. Abstract Similarity (from BERT)
    print(" Calculating abstract bert similarity....")
    df_copy['abstract_similarity'] = df_copy.apply(lambda row: calculate_abstract_similarity_bert(row, abstract_embeddings_matrix, pid_to_embed_idx), axis=1)

    # 3. Basic Graph Features
    print(" Calculating basic graph features...")
    df_copy['out_degree_p1'] = df_copy['paper_id_1'].map(out_deg_map).fillna(0)
    df_copy['in_degree_p2'] = df_copy['paper_id_2'].map(in_deg_map).fillna(0)
    df_copy['preferential_attachment'] = df_copy['out_degree_p1'] * df_copy['in_degree_p2']

    # 4. Advanced Graph Features
    print("Calculating advanced graph features...")
    
    # jaccard on successors
    def jaccard_successors(row):
        p1_cites = cited_by_lookup_map.get(row['paper_id_1'], set())
        
        p2_cites = cited_by_lookup_map.get(row['paper_id_2'], set())
        
        intersection = len(p1_cites.intersection(p2_cites))
        union = len(p1_cites.union(p2_cites))
        
        return intersection / union if union > 0 else 0.0
    df_copy['jaccard_successors'] = df_copy.apply(jaccard_successors, axis=1)

    # jaccard on pre-desesors
    def jaccard_predecessors(row):
        p1_cited_by = cites_lookup_map.get(row['paper_id_1'], set())
        p2_cited_by = cites_lookup_map.get(row['paper_id_2'], set())
        intersection = len(p1_cited_by.intersection(p2_cited_by))
        union = len(p1_cited_by.union(p2_cited_by))
        return intersection / union if union > 0 else 0.0
    df_copy['jaccard_predecessors'] = df_copy.apply(jaccard_predecessors, axis=1)
    
    # Adamic-Adar and Resource Allocation
    if nx_graph_undirected:
        pairs_to_predict = [(u, v) for u, v in df_copy[['paper_id_1', 'paper_id_2']].values if nx_graph_undirected.has_node(u) and nx_graph_undirected.has_node(v)]
        
        adamic_adar_preds = nx.adamic_adar_index(nx_graph_undirected, pairs_to_predict)
        resource_alloc_preds = nx.resource_allocation_index(nx_graph_undirected, pairs_to_predict)

        adamic_map = { (u, v): p for u, v, p in adamic_adar_preds }
        resource_map = { (u, v): p for u, v, p in resource_alloc_preds }

        df_copy['adamic_adar'] = df_copy.apply(lambda row: adamic_map.get((row['paper_id_1'], row['paper_id_2']), 0), axis=1)
        df_copy['resource_alloc'] = df_copy.apply(lambda row: resource_map.get((row['paper_id_1'], row['paper_id_2']), 0), axis=1)
    else:
        df_copy['adamic_adar'] = 0.0
        df_copy['resource_alloc'] = 0.0

    print(f"Feature engineering for '{df_name}' complete (total in specific this df: {time.time()-current_time:.2f}s).")
    return df_copy

# Apply Feature Engineering to Training Data
This cell calls the engineer_features function to process df_train DataFrame. After execution, df_train_features will contain the original training data along with all the newly engineered features. It then prints the head, NaN counts, and info for this augmented DataFrame.

In [21]:

df_train_features = pd.DataFrame() 
if not df_train.empty:
    df_train_features = engineer_features(
        df_train, 
        authors_dict,
        abstract_embeddings,
        pid_to_bert_idx,
        out_degrees,
        in_degrees,
        cited_by_map,
        cites_map,
        G_undirected
    )
    print("\nTraining data with features HEAD (first 5 rows):\n", df_train_features.head())




Engineering features for DataFrame 'df_train' with 2183910 rows...
 Calculating common authors...
 Calculating abstract bert similarity....
 Calculating basic graph features...
Calculating advanced graph features...
Feature engineering for 'df_train' complete (total in specific this df: 558.04s).

Training data with features HEAD (first 5 rows):
    paper_id_1  paper_id_2  target  common_authors  abstract_similarity  \
0       16646       73007       1               0             0.484608   
1       12798       44868       0               0             0.155915   
2        3212       17042       1               0             0.615255   
3      103632       71311       0               0             0.219759   
4       12509       71431       1               0             0.106953   

   out_degree_p1  in_degree_p2  preferential_attachment  jaccard_successors  \
0            2.0           7.0                     14.0                 0.0   
1            6.0          10.0                 

# Apply Feature Engineering to Kaggle Test Data
Similar to the previous cell, this one applies the engineer_features function to the Kaggle test set (df_test_kaggle). The resulting df_test_kaggle_features will be used later for making predictions. 

In [22]:
df_test_kaggle_features = pd.DataFrame() 
if not df_test_kaggle.empty:
    df_test_kaggle_features = engineer_features(
        df_test_kaggle, 
        authors_dict,
        abstract_embeddings,
        pid_to_bert_idx,
        out_degrees,
        in_degrees,
        cited_by_map,
        cites_map,
        G_undirected
    )
    print("\nKaggle test set with features HEAD(first 5 rows):\n", df_test_kaggle_features.head())


Engineering features for DataFrame 'df_test_kaggle' with 106692 rows...
 Calculating common authors...
 Calculating abstract bert similarity....
 Calculating basic graph features...
Calculating advanced graph features...
Feature engineering for 'df_test_kaggle' complete (total in specific this df: 27.33s).

Kaggle test set with features HEAD(first 5 rows):
    paper_id_1  paper_id_2  common_authors  abstract_similarity  out_degree_p1  \
0       34977       59394               0             0.290162           36.0   
1       22518       46602               0             0.062103            4.0   
2       36762       22813               0             0.423941           11.0   
3       44960      110384               0             0.118599           61.0   
4       29015       26366               0             0.212365           13.0   

   in_degree_p2  preferential_attachment  jaccard_successors  \
0          18.0                    648.0            0.020833   
1          16.0         

# Model Training and Prediction
This is the final cell where the machine learning model (LightGBM) is trained using the engineered features from df_train_features. It includes splitting the data for validation, training the model with early stopping, making predictions on the df_test_kaggle_features, and preparing the submission.csv file.

5-Fold Cross-Validation using a LightGBM (LGBM) classifier. The goal is to build a more struct model by training 5 slightly different versions of it and averaging their predictions together.

5 Folds trained with different data, produce one average result

In [23]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold


# 1. Define the 9  features to be used ahead
features_to_use = [
    'common_authors', 'abstract_similarity', 'out_degree_p1', 'in_degree_p2', 
    'preferential_attachment', 'jaccard_successors', 'jaccard_predecessors',
    'adamic_adar', 'resource_alloc'
]

if not df_train_features.empty:
    X = df_train_features[features_to_use]
    y = df_train_features['target']
    print(f"\nShape of X (training features): {X.shape}")

if not df_test_kaggle_features.empty:
    X_kaggle_test_final = df_test_kaggle_features[features_to_use]
    print(f"Shape of X_kaggle_test_final (Kaggle test features): {X_kaggle_test_final.shape}")


# 2. training with 5-Fold Cross-Validation

if not X.empty and not y.empty:
    print("\nStarting 5-Fold Cross-Validation Training...")
    N_SPLITS = 5
    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
    
    test_preds = np.zeros(len(X_kaggle_test_final))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\n===== Fold {fold+1} ======")
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        lgbm_model = lgb.LGBMClassifier(
            objective='binary', metric='logloss', n_estimators=2500,
            learning_rate=0.01, num_leaves=80, max_depth=10,   
            random_state=42 + fold, n_jobs=-1, colsample_bytree=0.7,
            subsample=0.7, reg_alpha=0.1, reg_lambda=0.1
        )
        
        lgbm_model.fit(X_train, y_train,
                       eval_set=[(X_val, y_val)],
                       eval_metric='logloss',
                       callbacks=[lgb.early_stopping(stopping_rounds=150, verbose=100)])
        
        test_preds += lgbm_model.predict_proba(X_kaggle_test_final)[:, 1] / N_SPLITS


# 3. Create -> csv
if len(test_preds) > 0:
    df_sample_sub = pd.read_csv(PATH_SAMPLE_SUBMISSION)
    df_submission = pd.DataFrame({'ID': df_sample_sub['ID'], 'Label': test_preds})
    df_submission.to_csv('submission.csv', index=False)
    print("\nSubmission file named 'submission.csv' created on Output.")
    print(df_submission.head())

print("\n - Script finished. - ")
print("\n Moving on sumbission if neseccary.")


Shape of X (training features): (2183910, 9)
Shape of X_kaggle_test_final (Kaggle test features): (106692, 9)

Starting 5-Fold Cross-Validation Training...

===== Fold 1 ======
[LightGBM] [Info] Number of positive: 873564, number of negative: 873564
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.059841 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1906
[LightGBM] [Info] Number of data points in the train set: 1747128, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 150 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth